# 🚀 Gradiente Descendente 2.0: Turbinando Nossa Descida na Montanha do Erro!

## Pedro Nunes Guth - Módulo 12/12: Cálculo para IA

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-12_img_01.png)

Fala pessoal! 🎉 Chegamos no último módulo do nosso curso de Cálculo para IA! Se você chegou até aqui, já entende que o gradiente descendente é tipo aquela bússola que nos guia pela montanha do erro. Mas tá, e se eu te disser que existem **versões turbinadas** dessa bússola?

No módulo anterior, aprendemos o gradiente descendente clássico - aquele marombeiro que pega TODOS os dados de uma vez pra calcular o gradiente. Hoje vamos conhecer os primos dele:

- **SGD (Stochastic Gradient Descent)**: O cara apressadinho que usa só UM exemplo por vez
- **Mini-batch**: O meio termo inteligente
- **Adam**: O algoritmo "espertão" que se adapta sozinho
- **E muito mais!**

Bora descer essa montanha com **ESTILO**! 🏔️⚡

In [ ]:
# Imports essenciais para nossa jornada!
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns
from sklearn.datasets import make_regression, make_classification
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configuração para gráficos mais bonitos
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Seed para reprodutibilidade
np.random.seed(42)

print("🎯 Ambiente preparado! Bora começar nossa aventura!")

## 🎯 1. Recapitulando: O Gradiente Descendente Clássico

Antes de partir para as variações, vamos relembrar nosso velho amigo do módulo anterior:

### A Fórmula Sagrada:

$$\theta_{novo} = \theta_{atual} - \alpha \nabla J(\theta)$$

Onde:
- $\theta$: nossos parâmetros (pesos do modelo)
- $\alpha$: taxa de aprendizado (learning rate)
- $\nabla J(\theta)$: o gradiente da função de custo

### O Problema do Gradiente Clássico:

No gradiente descendente **batch** (clássico), calculamos o gradiente usando **TODOS** os exemplos:

$$\nabla J(\theta) = \frac{1}{m} \sum_{i=1}^{m} \nabla J_i(\theta)$$

Tá, mas imagina que você tem 1 milhão de exemplos! Vai calcular o gradiente com TODOS eles a cada iteração? É como querer contar todos os grãos de areia da praia antes de dar um passo!



**Dica do Pedro**: O gradiente batch é preciso, mas LENTO. É tipo aquele amigo que demora 2 horas pra escolher o que vai comer no restaurante! 😅

In [ ]:
# Vamos criar um dataset para demonstrar os conceitos
def criar_dataset_exemplo(n_samples=1000, noise=0.1):
    """
    Cria um dataset simples para regressão linear
    y = 3x + 2 + ruído
    """
    X = np.random.randn(n_samples, 1)
    y = 3 * X.flatten() + 2 + noise * np.random.randn(n_samples)
    return X, y

# Função de custo MSE (Mean Squared Error)
def mse_loss(y_true, y_pred):
    """Calcula o erro quadrático médio"""
    return np.mean((y_true - y_pred) ** 2)

# Predição linear simples
def predict(X, w, b):
    """Faz predição: y = w*x + b"""
    return X.flatten() * w + b

# Implementação do Gradiente Descendente Batch (clássico)
def gradient_descent_batch(X, y, learning_rate=0.01, epochs=100):
    """Gradiente Descendente usando TODOS os dados por iteração"""
    # Inicialização dos parâmetros
    w = np.random.randn()
    b = np.random.randn()
    
    # Histórico para visualização
    history = {'loss': [], 'w': [], 'b': []}
    
    m = len(y)  # número de exemplos
    
    for epoch in range(epochs):
        # Predição usando TODOS os dados
        y_pred = predict(X, w, b)
        
        # Cálculo do erro
        loss = mse_loss(y, y_pred)
        
        # Cálculo dos gradientes usando TODOS os exemplos
        dw = -(2/m) * np.sum((y - y_pred) * X.flatten())
        db = -(2/m) * np.sum(y - y_pred)
        
        # Atualização dos parâmetros
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        # Salvar histórico
        history['loss'].append(loss)
        history['w'].append(w)
        history['b'].append(b)
    
    return w, b, history

# Testando o gradiente batch
X, y = criar_dataset_exemplo(1000)
w_batch, b_batch, hist_batch = gradient_descent_batch(X, y, learning_rate=0.1, epochs=50)

print(f"🎯 Gradiente Batch - Resultado final:")
print(f"   Peso (w): {w_batch:.3f} (esperado: ~3.0)")
print(f"   Bias (b): {b_batch:.3f} (esperado: ~2.0)")
print(f"   Loss final: {hist_batch['loss'][-1]:.6f}")

## 🏃‍♂️ 2. SGD: O Gradiente Descendente Apressadinho

Agora vem o **Stochastic Gradient Descent (SGD)**! Esse cara é tipo aquele amigo que decide na primeira opção do cardápio - ele usa apenas **UM** exemplo por vez para calcular o gradiente!

### A Matemática do SGD:

Em vez de usar todos os exemplos:
$$\nabla J(\theta) = \frac{1}{m} \sum_{i=1}^{m} \nabla J_i(\theta)$$

O SGD usa apenas um exemplo aleatório por iteração:
$$\nabla J(\theta) \approx \nabla J_i(\theta)$$

### Vantagens do SGD:
1. **Muito mais rápido**: Uma iteração é super rápida!
2. **Usa menos memória**: Processa um exemplo por vez
3. **Pode escapar de mínimos locais**: O "ruído" ajuda!

### Desvantagens:
1. **Convergência mais "bagunçada"**: O caminho não é suave
2. **Pode oscilar muito**: Como um marinheiro bêbado descendo a montanha! 🍺

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-12_img_03.png)

**Dica do Pedro**: O SGD é como dirigir no trânsito do Rio - você vai chegando no destino, mas fazendo umas curvas malucas pelo caminho! 🚗💨

In [ ]:
# Implementação do SGD (Stochastic Gradient Descent)
def stochastic_gradient_descent(X, y, learning_rate=0.01, epochs=100):
    """SGD - usa apenas UM exemplo por iteração"""
    # Inicialização
    w = np.random.randn()
    b = np.random.randn()
    
    # Histórico
    history = {'loss': [], 'w': [], 'b': []}
    
    m = len(y)
    
    for epoch in range(epochs):
        # Embaralha os índices para cada época
        indices = np.random.permutation(m)
        
        epoch_loss = 0
        
        # Para cada exemplo individual
        for i in indices:
            # Pega apenas UM exemplo
            xi = X[i:i+1]
            yi = y[i]
            
            # Predição para esse exemplo
            y_pred_i = predict(xi, w, b)
            
            # Gradientes baseados em apenas UM exemplo
            error = yi - y_pred_i
            dw = -2 * error * xi.flatten()[0]
            db = -2 * error
            
            # Atualização dos parâmetros
            w = w - learning_rate * dw
            b = b - learning_rate * db
            
            epoch_loss += (error ** 2)
        
        # Loss média da época
        avg_loss = epoch_loss / m
        history['loss'].append(avg_loss)
        history['w'].append(w)
        history['b'].append(b)
    
    return w, b, history

# Testando SGD
np.random.seed(42)  # Para comparação justa
w_sgd, b_sgd, hist_sgd = stochastic_gradient_descent(X, y, learning_rate=0.01, epochs=50)

print(f"⚡ SGD - Resultado final:")
print(f"   Peso (w): {w_sgd:.3f} (esperado: ~3.0)")
print(f"   Bias (b): {b_sgd:.3f} (esperado: ~2.0)")
print(f"   Loss final: {hist_sgd['loss'][-1]:.6f}")

print(f"\n📊 Comparação:")
print(f"   Batch GD Loss: {hist_batch['loss'][-1]:.6f}")
print(f"   SGD Loss: {hist_sgd['loss'][-1]:.6f}")

In [ ]:
# Visualizando a diferença entre Batch GD e SGD
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico 1: Convergência da Loss
axes[0,0].plot(hist_batch['loss'], label='Batch GD', linewidth=3, color='blue')
axes[0,0].plot(hist_sgd['loss'], label='SGD', linewidth=2, color='red', alpha=0.7)
axes[0,0].set_title('📉 Convergência da Função de Custo')
axes[0,0].set_xlabel('Épocas')
axes[0,0].set_ylabel('Loss (MSE)')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Gráfico 2: Evolução do peso W
axes[0,1].plot(hist_batch['w'], label='Batch GD', linewidth=3, color='blue')
axes[0,1].plot(hist_sgd['w'], label='SGD', linewidth=2, color='red', alpha=0.7)
axes[0,1].axhline(y=3.0, color='green', linestyle='--', label='Valor Real (3.0)')
axes[0,1].set_title('⚖️ Evolução do Peso (w)')
axes[0,1].set_xlabel('Épocas')
axes[0,1].set_ylabel('Valor do Peso')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Gráfico 3: Evolução do bias B
axes[1,0].plot(hist_batch['b'], label='Batch GD', linewidth=3, color='blue')
axes[1,0].plot(hist_sgd['b'], label='SGD', linewidth=2, color='red', alpha=0.7)
axes[1,0].axhline(y=2.0, color='green', linestyle='--', label='Valor Real (2.0)')
axes[1,0].set_title('📏 Evolução do Bias (b)')
axes[1,0].set_xlabel('Épocas')
axes[1,0].set_ylabel('Valor do Bias')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Gráfico 4: Trajetória no espaço de parâmetros
axes[1,1].plot(hist_batch['w'], hist_batch['b'], 'b-', linewidth=3, label='Batch GD', marker='o', markersize=4)
axes[1,1].plot(hist_sgd['w'][:20], hist_sgd['b'][:20], 'r-', linewidth=2, label='SGD (primeiras 20)', alpha=0.7, marker='s', markersize=3)
axes[1,1].plot(3.0, 2.0, 'g*', markersize=15, label='Ótimo Real')
axes[1,1].set_title('🎯 Trajetória no Espaço de Parâmetros')
axes[1,1].set_xlabel('Peso (w)')
axes[1,1].set_ylabel('Bias (b)')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🔍 Observações:")
print("   • Batch GD: Convergência suave e estável (linha azul)")
print("   • SGD: Convergência mais 'bagunçada', mas pode ser mais rápida (linha vermelha)")
print("   • SGD oscila mais, mas consegue escapar de mínimos locais!")

## 🎯 3. Mini-Batch: O Meio-Termo Inteligente

Tá, mas e se a gente pegasse o melhor dos dois mundos? O **Mini-Batch Gradient Descent** é exatamente isso!

Em vez de usar:
- **Todos** os exemplos (Batch) 🐌
- **Apenas um** exemplo (SGD) 🏃‍♂️💨

Usamos um **pequeno grupo** de exemplos por vez!

### A Matemática do Mini-Batch:

Para um mini-batch de tamanho $B$:
$$\nabla J(\theta) \approx \frac{1}{B} \sum_{i \in \text{mini-batch}} \nabla J_i(\theta)$$

### Tamanhos Típicos de Mini-Batch:
- **32**: Clássico para problemas pequenos
- **64**: Muito popular
- **128, 256**: Para datasets maiores
- **512+**: Para datasets gigantes

### Vantagens do Mini-Batch:
1. **Mais estável que SGD**: Menos oscilação
2. **Mais rápido que Batch**: Não precisa de todos os dados
3. **Vetorização eficiente**: GPUs adoram mini-batches!
4. **Melhor generalização**: Ruído controlado ajuda

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-12_img_04.png)

**Dica do Pedro**: Mini-batch é como pedir uma porção individual no restaurante em vez de um prato pra família toda, mas também não é só um salgadinho! É o equilíbrio perfeito! 🍽️

In [ ]:
# Implementação do Mini-Batch Gradient Descent
def mini_batch_gradient_descent(X, y, batch_size=32, learning_rate=0.01, epochs=100):
    """Mini-Batch GD - usa pequenos grupos de exemplos"""
    # Inicialização
    w = np.random.randn()
    b = np.random.randn()
    
    # Histórico
    history = {'loss': [], 'w': [], 'b': []}
    
    m = len(y)
    
    for epoch in range(epochs):
        # Embaralha os dados
        indices = np.random.permutation(m)
        X_shuffled = X[indices]
        y_shuffled = y[indices]
        
        epoch_loss = 0
        num_batches = 0
        
        # Processa em mini-batches
        for i in range(0, m, batch_size):
            # Pega o mini-batch
            end_idx = min(i + batch_size, m)
            X_batch = X_shuffled[i:end_idx]
            y_batch = y_shuffled[i:end_idx]
            
            # Predição para o mini-batch
            y_pred_batch = predict(X_batch, w, b)
            
            # Loss do mini-batch
            batch_loss = mse_loss(y_batch, y_pred_batch)
            epoch_loss += batch_loss
            
            # Gradientes do mini-batch
            batch_m = len(y_batch)
            dw = -(2/batch_m) * np.sum((y_batch - y_pred_batch) * X_batch.flatten())
            db = -(2/batch_m) * np.sum(y_batch - y_pred_batch)
            
            # Atualização dos parâmetros
            w = w - learning_rate * dw
            b = b - learning_rate * db
            
            num_batches += 1
        
        # Loss média da época
        avg_loss = epoch_loss / num_batches
        history['loss'].append(avg_loss)
        history['w'].append(w)
        history['b'].append(b)
    
    return w, b, history

# Testando diferentes tamanhos de mini-batch
batch_sizes = [8, 32, 128]
results = {}

print("🧪 Testando diferentes tamanhos de mini-batch...\n")

for batch_size in batch_sizes:
    np.random.seed(42)  # Para comparação justa
    w_mb, b_mb, hist_mb = mini_batch_gradient_descent(
        X, y, batch_size=batch_size, learning_rate=0.05, epochs=50
    )
    
    results[batch_size] = (w_mb, b_mb, hist_mb)
    
    print(f"📦 Mini-Batch (size={batch_size}):")
    print(f"   Peso (w): {w_mb:.3f}")
    print(f"   Bias (b): {b_mb:.3f}")
    print(f"   Loss final: {hist_mb['loss'][-1]:.6f}\n")

In [ ]:
# Comparando os diferentes métodos
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Convergência da Loss
axes[0].plot(hist_batch['loss'], label='Batch GD', linewidth=3, color='blue')
axes[0].plot(hist_sgd['loss'], label='SGD', linewidth=2, color='red', alpha=0.8)

colors = ['green', 'orange', 'purple']
for i, batch_size in enumerate(batch_sizes):
    _, _, hist = results[batch_size]
    axes[0].plot(hist['loss'], label=f'Mini-Batch ({batch_size})', 
                linewidth=2, color=colors[i], alpha=0.8)

axes[0].set_title('📈 Comparação de Convergência')
axes[0].set_xlabel('Épocas')
axes[0].set_ylabel('Loss (MSE)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')  # Escala logarítmica para melhor visualização

# Gráfico 2: Estabilidade (últimas 10 épocas)
last_n = 10
batch_std = np.std(hist_batch['loss'][-last_n:])
sgd_std = np.std(hist_sgd['loss'][-last_n:])

methods = ['Batch GD', 'SGD']
stds = [batch_std, sgd_std]
colors_bar = ['blue', 'red']

for i, batch_size in enumerate(batch_sizes):
    _, _, hist = results[batch_size]
    mb_std = np.std(hist['loss'][-last_n:])
    methods.append(f'Mini-Batch ({batch_size})')
    stds.append(mb_std)
    colors_bar.append(colors[i])

bars = axes[1].bar(methods, stds, color=colors_bar, alpha=0.7)
axes[1].set_title('📊 Estabilidade (Desvio Padrão das Últimas 10 Épocas)')
axes[1].set_ylabel('Desvio Padrão da Loss')
axes[1].tick_params(axis='x', rotation=45)

# Adiciona valores nas barras
for bar, std in zip(bars, stds):
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{std:.6f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("🎯 Análise dos Resultados:")
print(f"   • Batch GD: Mais estável (std={batch_std:.6f}), mas pode ser mais lento")
print(f"   • SGD: Menos estável (std={sgd_std:.6f}), mas mais rápido")
print("   • Mini-Batches: Equilibram velocidade e estabilidade!")

## ⚡ 4. A Taxa de Aprendizado: O Acelerador da Nossa Descida

Agora vamos falar de um dos **parâmetros mais críticos** de qualquer algoritmo de otimização: a **taxa de aprendizado** ($\alpha$)!

### O Que É a Taxa de Aprendizado?

Lembra da nossa fórmula sagrada?
$$\theta_{novo} = \theta_{atual} - \alpha \nabla J(\theta)$$

O $\alpha$ é como o **"tamanho do passo"** que damos na descida da montanha!

### Os Três Cenários Clássicos:

1. **$\alpha$ muito pequeno** (ex: 0.001):
   - ✅ Convergência estável
   - ❌ **MUITO lento** - como uma lesma subindo no vidro!
   - ❌ Pode travar em platôs

2. **$\alpha$ muito grande** (ex: 1.0):
   - ✅ Convergência rápida (no início)
   - ❌ **Pode divergir** - como um carro sem freio!
   - ❌ Oscila em volta do mínimo sem nunca chegar

3. **$\alpha$ na medida certa** (ex: 0.01-0.1):
   - ✅ Convergência rápida E estável
   - ✅ Equilíbrio perfeito! 🎯

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-12_img_05.png)

**Dica do Pedro**: Escolher a taxa de aprendizado é como ajustar a velocidade no Mario Kart - muito devagar você perde a corrida, muito rápido você bate na parede! 🏎️💨

In [ ]:
# Testando diferentes taxas de aprendizado
learning_rates = [0.001, 0.01, 0.1, 0.5, 1.0]
lr_results = {}

print("🎛️ Testando diferentes taxas de aprendizado...\n")

for lr in learning_rates:
    try:
        np.random.seed(42)
        w_lr, b_lr, hist_lr = mini_batch_gradient_descent(
            X, y, batch_size=32, learning_rate=lr, epochs=100
        )
        
        lr_results[lr] = (w_lr, b_lr, hist_lr)
        
        # Verifica se convergiu (loss final < 1000 - threshold arbitrário)
        converged = hist_lr['loss'][-1] < 1000
        status = "✅ Convergiu" if converged else "❌ Divergiu"
        
        print(f"📊 Learning Rate = {lr}:")
        print(f"   Status: {status}")
        print(f"   Loss final: {hist_lr['loss'][-1]:.6f}")
        print(f"   Peso final: {w_lr:.3f}")
        print(f"   Bias final: {b_lr:.3f}\n")
        
    except Exception as e:
        print(f"💥 Learning Rate = {lr}: EXPLODIU! (Overflow)\n")
        lr_results[lr] = None

In [ ]:
# Visualizando o efeito das diferentes taxas de aprendizado
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Cores para cada learning rate
colors = ['blue', 'green', 'orange', 'red', 'purple']

# Gráfico 1: Convergência da Loss (escala normal)
for i, lr in enumerate(learning_rates):
    if lr_results[lr] is not None:
        w, b, hist = lr_results[lr]
        # Plota apenas se a loss não explodiu
        if max(hist['loss']) < 1000:
            axes[0,0].plot(hist['loss'], label=f'LR = {lr}', 
                          color=colors[i], linewidth=2)

axes[0,0].set_title('📉 Convergência da Loss (Escala Normal)')
axes[0,0].set_xlabel('Épocas')
axes[0,0].set_ylabel('Loss (MSE)')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Gráfico 2: Convergência da Loss (escala log)
for i, lr in enumerate(learning_rates):
    if lr_results[lr] is not None:
        w, b, hist = lr_results[lr]
        if max(hist['loss']) < 1000:
            axes[0,1].semilogy(hist['loss'], label=f'LR = {lr}', 
                              color=colors[i], linewidth=2)

axes[0,1].set_title('📉 Convergência da Loss (Escala Log)')
axes[0,1].set_xlabel('Épocas')
axes[0,1].set_ylabel('Loss (MSE) - Log Scale')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Gráfico 3: Evolução dos pesos
for i, lr in enumerate(learning_rates):
    if lr_results[lr] is not None:
        w, b, hist = lr_results[lr]
        if max(hist['loss']) < 1000:
            axes[1,0].plot(hist['w'], label=f'LR = {lr}', 
                          color=colors[i], linewidth=2)

axes[1,0].axhline(y=3.0, color='black', linestyle='--', alpha=0.5, label='Valor Real')
axes[1,0].set_title('⚖️ Evolução do Peso (w)')
axes[1,0].set_xlabel('Épocas')
axes[1,0].set_ylabel('Valor do Peso')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Gráfico 4: Análise de estabilidade (últimas 20 épocas)
stable_lrs = []
volatilities = []

for lr in learning_rates:
    if lr_results[lr] is not None:
        w, b, hist = lr_results[lr]
        if max(hist['loss']) < 1000:
            # Calcular volatilidade das últimas 20 épocas
            last_losses = hist['loss'][-20:]
            volatility = np.std(last_losses)
            stable_lrs.append(lr)
            volatilities.append(volatility)

bars = axes[1,1].bar([str(lr) for lr in stable_lrs], volatilities, 
                    color=[colors[learning_rates.index(lr)] for lr in stable_lrs], alpha=0.7)
axes[1,1].set_title('📊 Estabilidade por Learning Rate')
axes[1,1].set_xlabel('Learning Rate')
axes[1,1].set_ylabel('Volatilidade (Desvio Padrão)')

# Adiciona valores nas barras
for bar, vol in zip(bars, volatilities):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                  f'{vol:.4f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\n🎯 Lições Aprendidas sobre Learning Rate:")
print("   • LR muito baixo (0.001): Lento mas estável")
print("   • LR médio (0.01-0.1): Sweet spot! 🎯")
print("   • LR muito alto (0.5+): Instável ou diverge")
print("\n💡 Dica: Comece com 0.01 e ajuste conforme necessário!")

## 🤖 5. Adam: O Algoritmo "Espertão" que Se Adapta Sozinho

Agora chegou a hora de conhecer a **estrela dos algoritmos de otimização**: o **Adam** (Adaptive Moment Estimation)!

O Adam é como aquele GPS inteligente que:
1. **Se adapta ao trânsito** (ajusta a taxa de aprendizado automaticamente)
2. **Lembra do caminho** (usa momentum das iterações passadas)
3. **Considera o terreno** (normaliza por gradientes passados)

### A Matemática do Adam:

O Adam combina duas ideias geniais:

**1. Momentum (memória das direções):**
$$m_t = \beta_1 m_{t-1} + (1-\beta_1) g_t$$

**2. RMSprop (normalização adaptativa):**
$$v_t = \beta_2 v_{t-1} + (1-\beta_2) g_t^2$$

**3. Correção de bias:**
$$\hat{m}_t = \frac{m_t}{1-\beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1-\beta_2^t}$$

**4. Atualização final:**
$$\theta_{t+1} = \theta_t - \frac{\alpha}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t$$

### Parâmetros Padrão do Adam:
- $\alpha = 0.001$ (learning rate)
- $\beta_1 = 0.9$ (momentum)
- $\beta_2 = 0.999$ (RMSprop)
- $\epsilon = 10^{-8}$ (estabilidade numérica)

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-12_img_06.png)

**Dica do Pedro**: Adam é como ter um personal trainer, um GPS e um meteorologista te ajudando ao mesmo tempo na descida da montanha! 🏋️‍♂️🗺️🌤️

In [ ]:
# Implementação do Adam Optimizer
class AdamOptimizer:
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        
        # Momentos para cada parâmetro
        self.m_w = 0  # primeiro momento para w
        self.v_w = 0  # segundo momento para w
        self.m_b = 0  # primeiro momento para b
        self.v_b = 0  # segundo momento para b
        
        self.t = 0    # contador de iterações
    
    def update(self, w, b, dw, db):
        """Atualiza os parâmetros usando Adam"""
        self.t += 1
        
        # Atualiza momentos para w
        self.m_w = self.beta1 * self.m_w + (1 - self.beta1) * dw
        self.v_w = self.beta2 * self.v_w + (1 - self.beta2) * (dw ** 2)
        
        # Atualiza momentos para b
        self.m_b = self.beta1 * self.m_b + (1 - self.beta1) * db
        self.v_b = self.beta2 * self.v_b + (1 - self.beta2) * (db ** 2)
        
        # Correção de bias
        m_w_corr = self.m_w / (1 - self.beta1 ** self.t)
        v_w_corr = self.v_w / (1 - self.beta2 ** self.t)
        m_b_corr = self.m_b / (1 - self.beta1 ** self.t)
        v_b_corr = self.v_b / (1 - self.beta2 ** self.t)
        
        # Atualização dos parâmetros
        w_new = w - self.lr * m_w_corr / (np.sqrt(v_w_corr) + self.epsilon)
        b_new = b - self.lr * m_b_corr / (np.sqrt(v_b_corr) + self.epsilon)
        
        return w_new, b_new

def gradient_descent_adam(X, y, batch_size=32, learning_rate=0.001, epochs=100):
    """Gradiente Descendente com Adam Optimizer"""
    # Inicialização
    w = np.random.randn()
    b = np.random.randn()
    
    # Inicializa o otimizador Adam
    optimizer = AdamOptimizer(learning_rate=learning_rate)
    
    # Histórico
    history = {'loss': [], 'w': [], 'b': []}
    
    m = len(y)
    
    for epoch in range(epochs):
        # Embaralha os dados
        indices = np.random.permutation(m)
        X_shuffled = X[indices]
        y_shuffled = y[indices]
        
        epoch_loss = 0
        num_batches = 0
        
        # Processa em mini-batches
        for i in range(0, m, batch_size):
            end_idx = min(i + batch_size, m)
            X_batch = X_shuffled[i:end_idx]
            y_batch = y_shuffled[i:end_idx]
            
            # Predição e loss
            y_pred_batch = predict(X_batch, w, b)
            batch_loss = mse_loss(y_batch, y_pred_batch)
            epoch_loss += batch_loss
            
            # Gradientes
            batch_m = len(y_batch)
            dw = -(2/batch_m) * np.sum((y_batch - y_pred_batch) * X_batch.flatten())
            db = -(2/batch_m) * np.sum(y_batch - y_pred_batch)
            
            # Atualização com Adam
            w, b = optimizer.update(w, b, dw, db)
            
            num_batches += 1
        
        # Salva histórico
        avg_loss = epoch_loss / num_batches
        history['loss'].append(avg_loss)
        history['w'].append(w)
        history['b'].append(b)
    
    return w, b, history

# Testando Adam
np.random.seed(42)
w_adam, b_adam, hist_adam = gradient_descent_adam(X, y, batch_size=32, learning_rate=0.01, epochs=50)

print("🤖 Adam Optimizer - Resultados:")
print(f"   Peso (w): {w_adam:.3f} (esperado: ~3.0)")
print(f"   Bias (b): {b_adam:.3f} (esperado: ~2.0)")
print(f"   Loss final: {hist_adam['loss'][-1]:.6f}")

```mermaid
graph TD
    A[Gradiente Atual] --> B[Momentum<br/>β₁ = 0.9]
    A --> C[RMSprop<br/>β₂ = 0.999]
    B --> D[Correção de Bias]
    C --> E[Correção de Bias]
    D --> F[Combinação Adam]
    E --> F
    F --> G[Atualização<br/>Adaptativa]
    G --> H[Novos Parâmetros]
```

In [ ]:
# Comparação final: SGD vs Mini-Batch vs Adam
methods = {
    'SGD': hist_sgd,
    'Mini-Batch (32)': results[32][2],
    'Adam': hist_adam
}

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

colors = ['red', 'green', 'blue']

# Gráfico 1: Convergência da Loss
for i, (method, hist) in enumerate(methods.items()):
    axes[0,0].plot(hist['loss'], label=method, linewidth=2.5, color=colors[i])

axes[0,0].set_title('🏁 Comparação Final: Convergência da Loss')
axes[0,0].set_xlabel('Épocas')
axes[0,0].set_ylabel('Loss (MSE)')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)
axes[0,0].set_yscale('log')

# Gráfico 2: Convergência do peso
for i, (method, hist) in enumerate(methods.items()):
    axes[0,1].plot(hist['w'], label=method, linewidth=2.5, color=colors[i])

axes[0,1].axhline(y=3.0, color='black', linestyle='--', alpha=0.5, label='Valor Real')
axes[0,1].set_title('⚖️ Convergência do Peso (w)')
axes[0,1].set_xlabel('Épocas')
axes[0,1].set_ylabel('Valor do Peso')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Gráfico 3: Velocidade de convergência (primeiras 20 épocas)
for i, (method, hist) in enumerate(methods.items()):
    axes[1,0].plot(hist['loss'][:20], label=method, linewidth=2.5, color=colors[i], marker='o', markersize=4)

axes[1,0].set_title('⚡ Velocidade Inicial (Primeiras 20 Épocas)')
axes[1,0].set_xlabel('Épocas')
axes[1,0].set_ylabel('Loss (MSE)')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Gráfico 4: Métricas finais
final_losses = []
method_names = []
final_errors_w = []  # Erro em relação ao valor real (3.0)

for method, hist in methods.items():
    method_names.append(method)
    final_losses.append(hist['loss'][-1])
    final_errors_w.append(abs(hist['w'][-1] - 3.0))  # Erro absoluto do peso

x = np.arange(len(method_names))
width = 0.35

# Normaliza as métricas para comparação
norm_losses = np.array(final_losses) / max(final_losses)
norm_errors = np.array(final_errors_w) / max(final_errors_w)

bars1 = axes[1,1].bar(x - width/2, norm_losses, width, label='Loss Final (Normalizada)', color='skyblue', alpha=0.8)
bars2 = axes[1,1].bar(x + width/2, norm_errors, width, label='Erro do Peso (Normalizado)', color='lightcoral', alpha=0.8)

axes[1,1].set_title('📊 Métricas Finais (Normalizadas)')
axes[1,1].set_ylabel('Valor Normalizado')
axes[1,1].set_xticks(x)
axes[1,1].set_xticklabels(method_names)
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

# Adiciona valores reais nas barras
for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
    # Loss
    height1 = bar1.get_height()
    axes[1,1].text(bar1.get_x() + bar1.get_width()/2., height1 + 0.01,
                  f'{final_losses[i]:.4f}', ha='center', va='bottom', fontsize=8)
    # Erro do peso
    height2 = bar2.get_height()
    axes[1,1].text(bar2.get_x() + bar2.get_width()/2., height2 + 0.01,
                  f'{final_errors_w[i]:.4f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

print("\n🏆 Ranking dos Algoritmos:")
print("\n📈 Por Velocidade de Convergência:")
print("   1. 🥇 Adam - Rápido e estável")
print("   2. 🥈 Mini-Batch - Bom equilíbrio")
print("   3. 🥉 SGD - Rápido mas instável")

print("\n🎯 Por Precisão Final:")
for i, method in enumerate(method_names):
    print(f"   {i+1}. {method}: Loss = {final_losses[i]:.6f}, Erro peso = {final_errors_w[i]:.4f}")

## 🧠 6. Outros Algoritmos: A Família Completa

Além do Adam, existem outros membros da família dos otimizadores adaptativos. Cada um tem sua personalidade!

### 🏃‍♂️ Momentum:
- **Ideia**: "Lembra" da direção anterior
- **Fórmula**: $v_t = \gamma v_{t-1} + \alpha \nabla J(\theta)$
- **Personalidade**: Como uma bola rolando ladeira abaixo - ganha velocidade!

### 📐 RMSprop:
- **Ideia**: Adapta a taxa de aprendizado por parâmetro
- **Fórmula**: $E[g^2]_t = \gamma E[g^2]_{t-1} + (1-\gamma) g_t^2$
- **Personalidade**: Como um GPS que ajusta a velocidade conforme o terreno

### ⚡ AdaGrad:
- **Ideia**: Diminui a taxa para parâmetros frequentemente atualizados
- **Problema**: Pode "morrer" (taxa fica muito pequena)
- **Personalidade**: Workaholic que se cansa rápido

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-12_img_07.png)

**Dica do Pedro**: Na prática, Adam é o "pau pra toda obra" - funciona bem na maioria dos casos. É como ter um canivete suíço dos otimizadores! 🔧

In [ ]:
# Implementação simples de outros otimizadores para comparação

class MomentumOptimizer:
    def __init__(self, learning_rate=0.01, momentum=0.9):
        self.lr = learning_rate
        self.momentum = momentum
        self.v_w = 0
        self.v_b = 0
    
    def update(self, w, b, dw, db):
        # Atualiza velocidades com momentum
        self.v_w = self.momentum * self.v_w + self.lr * dw
        self.v_b = self.momentum * self.v_b + self.lr * db
        
        # Atualiza parâmetros
        w_new = w - self.v_w
        b_new = b - self.v_b
        
        return w_new, b_new

class RMSpropOptimizer:
    def __init__(self, learning_rate=0.001, decay=0.9, epsilon=1e-8):
        self.lr = learning_rate
        self.decay = decay
        self.epsilon = epsilon
        self.s_w = 0
        self.s_b = 0
    
    def update(self, w, b, dw, db):
        # Atualiza médias móveis dos gradientes ao quadrado
        self.s_w = self.decay * self.s_w + (1 - self.decay) * (dw ** 2)
        self.s_b = self.decay * self.s_b + (1 - self.decay) * (db ** 2)
        
        # Atualiza parâmetros
        w_new = w - self.lr * dw / (np.sqrt(self.s_w) + self.epsilon)
        b_new = b - self.lr * db / (np.sqrt(self.s_b) + self.epsilon)
        
        return w_new, b_new

def test_optimizer(optimizer_class, optimizer_name, **kwargs):
    """Testa um otimizador específico"""
    np.random.seed(42)
    
    # Inicialização
    w = np.random.randn()
    b = np.random.randn()
    optimizer = optimizer_class(**kwargs)
    
    # Histórico
    history = {'loss': [], 'w': [], 'b': []}
    
    batch_size = 32
    epochs = 50
    m = len(y)
    
    for epoch in range(epochs):
        indices = np.random.permutation(m)
        X_shuffled = X[indices]
        y_shuffled = y[indices]
        
        epoch_loss = 0
        num_batches = 0
        
        for i in range(0, m, batch_size):
            end_idx = min(i + batch_size, m)
            X_batch = X_shuffled[i:end_idx]
            y_batch = y_shuffled[i:end_idx]
            
            y_pred_batch = predict(X_batch, w, b)
            batch_loss = mse_loss(y_batch, y_pred_batch)
            epoch_loss += batch_loss
            
            batch_m = len(y_batch)
            dw = -(2/batch_m) * np.sum((y_batch - y_pred_batch) * X_batch.flatten())
            db = -(2/batch_m) * np.sum(y_batch - y_pred_batch)
            
            w, b = optimizer.update(w, b, dw, db)
            num_batches += 1
        
        avg_loss = epoch_loss / num_batches
        history['loss'].append(avg_loss)
        history['w'].append(w)
        history['b'].append(b)
    
    return w, b, history

# Testando todos os otimizadores
optimizers_results = {}

print("🧪 Testando a família completa de otimizadores...\n")

# SGD simples (já temos)
optimizers_results['SGD'] = (w_sgd, b_sgd, hist_sgd)

# Momentum
w_mom, b_mom, hist_mom = test_optimizer(MomentumOptimizer, 'Momentum', learning_rate=0.01, momentum=0.9)
optimizers_results['Momentum'] = (w_mom, b_mom, hist_mom)

# RMSprop
w_rms, b_rms, hist_rms = test_optimizer(RMSpropOptimizer, 'RMSprop', learning_rate=0.01, decay=0.9)
optimizers_results['RMSprop'] = (w_rms, b_rms, hist_rms)

# Adam (já temos)
optimizers_results['Adam'] = (w_adam, b_adam, hist_adam)

# Resultados
for name, (w_final, b_final, hist) in optimizers_results.items():
    print(f"🤖 {name}:")
    print(f"   Peso final: {w_final:.3f}")
    print(f"   Bias final: {b_final:.3f}")
    print(f"   Loss final: {hist['loss'][-1]:.6f}\n")

In [ ]:
# Comparação visual de todos os otimizadores
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

colors = ['red', 'orange', 'green', 'blue']
opt_names = list(optimizers_results.keys())

# Gráfico 1: Convergência da Loss
for i, (name, (_, _, hist)) in enumerate(optimizers_results.items()):
    axes[0,0].plot(hist['loss'], label=name, linewidth=2.5, color=colors[i])

axes[0,0].set_title('🏆 Batalha dos Otimizadores: Convergência da Loss')
axes[0,0].set_xlabel('Épocas')
axes[0,0].set_ylabel('Loss (MSE)')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)
axes[0,0].set_yscale('log')

# Gráfico 2: Trajetória no espaço de parâmetros
for i, (name, (_, _, hist)) in enumerate(optimizers_results.items()):
    # Plota apenas os primeiros 30 pontos para clareza
    axes[0,1].plot(hist['w'][:30], hist['b'][:30], label=name, linewidth=2, 
                  color=colors[i], marker='o', markersize=3, alpha=0.8)

axes[0,1].plot(3.0, 2.0, 'k*', markersize=15, label='Ótimo Real')
axes[0,1].set_title('🎯 Trajetórias no Espaço de Parâmetros')
axes[0,1].set_xlabel('Peso (w)')
axes[0,1].set_ylabel('Bias (b)')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Gráfico 3: Velocidade de convergência (primeiras 15 épocas)
for i, (name, (_, _, hist)) in enumerate(optimizers_results.items()):
    axes[1,0].plot(hist['loss'][:15], label=name, linewidth=2.5, 
                  color=colors[i], marker='s', markersize=4)

axes[1,0].set_title('⚡ Largada: Primeiras 15 Épocas')
axes[1,0].set_xlabel('Épocas')
axes[1,0].set_ylabel('Loss (MSE)')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)
axes[1,0].set_yscale('log')

# Gráfico 4: Ranking final
final_losses = [hist['loss'][-1] for _, (_, _, hist) in optimizers_results.items()]
final_w_errors = [abs(w_final - 3.0) for w_final, _, _ in optimizers_results.values()]

# Calcula pontuação (menor é melhor)
loss_ranks = np.argsort(final_losses) + 1
error_ranks = np.argsort(final_w_errors) + 1
total_scores = loss_ranks + error_ranks  # Menor pontuação = melhor

# Ordena por pontuação total
sorted_indices = np.argsort(total_scores)
sorted_names = [opt_names[i] for i in sorted_indices]
sorted_scores = [total_scores[i] for i in sorted_indices]

# Cores baseadas na posição
rank_colors = ['gold', 'silver', '#CD7F32', 'gray']  # Ouro, Prata, Bronze, Cinza

bars = axes[1,1].bar(sorted_names, sorted_scores, color=rank_colors, alpha=0.8)
axes[1,1].set_title('🏅 Ranking Final (Menor = Melhor)')
axes[1,1].set_ylabel('Pontuação Total')
axes[1,1].set_xlabel('Otimizador')

# Adiciona medalhas
medals = ['🥇', '🥈', '🥉', '4th']
for i, (bar, score) in enumerate(zip(bars, sorted_scores)):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height + 0.05,
                  f'{medals[i]}\n{score}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🏆 RANKING FINAL DOS OTIMIZADORES:")
for i, idx in enumerate(sorted_indices):
    name = opt_names[idx]
    loss = final_losses[idx]
    w_error = final_w_errors[idx]
    print(f"   {medals[i]} {name}: Loss={loss:.6f}, Erro_peso={w_error:.4f}")

print("\n💡 Resumo das Personalidades:")
print("   • SGD: Rápido mas instável (como um carro de corrida sem controle)")
print("   • Momentum: Ganha velocidade com o tempo (como uma bola descendo)")
print("   • RMSprop: Se adapta ao terreno (como um veículo todo-terreno)")
print("   • Adam: O equilibrado (como um carro de luxo com todos os recursos)")

## 🛠️ 7. Exercício Prático: Criando Seu Próprio Otimizador

Agora é sua vez de brilhar! Vamos criar um desafio prático para consolidar todo o conhecimento.

### 🎯 Desafio 1: Implemente o AdaMax

O **AdaMax** é uma variação do Adam que usa a norma infinita em vez da norma L2. Sua fórmula é:

$$m_t = \beta_1 m_{t-1} + (1-\beta_1) g_t$$
$$u_t = \max(\beta_2 u_{t-1}, |g_t|)$$
$$\theta_{t+1} = \theta_t - \frac{\alpha}{u_t} m_t$$

**Sua missão**: Complete a implementação abaixo!

In [ ]:
# EXERCÍCIO 1: Implemente o AdaMax
class AdaMaxOptimizer:
    def __init__(self, learning_rate=0.002, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        
        # TODO: Inicialize os momentos
        self.m_w = 0  # primeiro momento para w
        self.u_w = 0  # norma infinita para w
        self.m_b = 0  # primeiro momento para b  
        self.u_b = 0  # norma infinita para b
        
        self.t = 0    # contador
    
    def update(self, w, b, dw, db):
        """TODO: Implemente a atualização do AdaMax"""
        self.t += 1
        
        # TODO: Atualize os momentos para w
        self.m_w = self.beta1 * self.m_w + (1 - self.beta1) * dw
        self.u_w = max(self.beta2 * self.u_w, abs(dw))
        
        # TODO: Atualize os momentos para b
        self.m_b = self.beta1 * self.m_b + (1 - self.beta1) * db
        self.u_b = max(self.beta2 * self.u_b, abs(db))
        
        # TODO: Correção de bias apenas para m (não para u!)
        m_w_corr = self.m_w / (1 - self.beta1 ** self.t)
        m_b_corr = self.m_b / (1 - self.beta1 ** self.t)
        
        # TODO: Atualização dos parâmetros
        w_new = w - self.lr * m_w_corr / (self.u_w + self.epsilon)
        b_new = b - self.lr * m_b_corr / (self.u_b + self.epsilon)
        
        return w_new, b_new

# Teste seu AdaMax!
print("🧪 Testando seu AdaMax...")
try:
    w_adamax, b_adamax, hist_adamax = test_optimizer(
        AdaMaxOptimizer, 'AdaMax', learning_rate=0.01, beta1=0.9, beta2=0.999
    )
    
    print(f"✅ AdaMax funcionou!")
    print(f"   Peso final: {w_adamax:.3f}")
    print(f"   Bias final: {b_adamax:.3f}")
    print(f"   Loss final: {hist_adamax['loss'][-1]:.6f}")
    
    # Comparação rápida com Adam
    print(f"\n📊 Comparação com Adam:")
    print(f"   Adam Loss: {hist_adam['loss'][-1]:.6f}")
    print(f"   AdaMax Loss: {hist_adamax['loss'][-1]:.6f}")
    
    if hist_adamax['loss'][-1] < hist_adam['loss'][-1]:
        print("   🎉 Seu AdaMax está melhor que o Adam!")
    else:
        print("   🤖 Adam ainda está na frente, mas bom trabalho!")
        
except Exception as e:
    print(f"❌ Ops! Algo deu errado: {e}")
    print("💡 Dica: Verifique se você implementou todas as partes TODO")

### 🎯 Desafio 2: Experimento com Learning Rate Schedule

Uma técnica avançada é **variar a taxa de aprendizado** durante o treinamento. Implemente um **Learning Rate Decay**:

$$\alpha_t = \alpha_0 \cdot \frac{1}{1 + \text{decay} \cdot t}$$

Onde $t$ é o número da iteração atual.

In [ ]:
# EXERCÍCIO 2: SGD com Learning Rate Decay
def sgd_with_lr_decay(X, y, initial_lr=0.1, decay=0.01, epochs=100):
    """SGD com taxa de aprendizado que diminui ao longo do tempo"""
    
    # TODO: Complete a implementação
    w = np.random.randn()
    b = np.random.randn()
    
    history = {'loss': [], 'w': [], 'b': [], 'lr': []}  # Adicionamos lr ao histórico
    m = len(y)
    
    for epoch in range(epochs):
        # TODO: Calcule a taxa de aprendizado atual
        current_lr = initial_lr / (1 + decay * epoch)
        
        # Embaralha dados
        indices = np.random.permutation(m)
        epoch_loss = 0
        
        # SGD para cada exemplo
        for i in indices:
            xi = X[i:i+1]
            yi = y[i]
            
            # TODO: Complete o SGD usando current_lr
            y_pred_i = predict(xi, w, b)
            error = yi - y_pred_i
            
            dw = -2 * error * xi.flatten()[0]
            db = -2 * error
            
            # Atualização com taxa decrescente
            w = w - current_lr * dw
            b = b - current_lr * db
            
            epoch_loss += (error ** 2)
        
        # Salva histórico
        avg_loss = epoch_loss / m
        history['loss'].append(avg_loss)
        history['w'].append(w)
        history['b'].append(b)
        history['lr'].append(current_lr)
    
    return w, b, history

# Testando diferentes configurações de decay
print("🧪 Testando Learning Rate Decay...\n")

decay_configs = [0.0, 0.01, 0.05, 0.1]  # 0.0 = sem decay
decay_results = {}

for decay in decay_configs:
    np.random.seed(42)
    w_decay, b_decay, hist_decay = sgd_with_lr_decay(
        X, y, initial_lr=0.1, decay=decay, epochs=50
    )
    
    decay_results[decay] = (w_decay, b_decay, hist_decay)
    
    print(f"📉 Decay = {decay}:")
    print(f"   Loss final: {hist_decay['loss'][-1]:.6f}")
    print(f"   LR inicial: {hist_decay['lr'][0]:.4f}")
    print(f"   LR final: {hist_decay['lr'][-1]:.4f}\n")

# Visualização do Learning Rate Decay
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

colors = ['blue', 'green', 'orange', 'red']

# Gráfico 1: Evolução da taxa de aprendizado
for i, decay in enumerate(decay_configs):
    _, _, hist = decay_results[decay]
    axes[0].plot(hist['lr'], label=f'Decay = {decay}', 
                linewidth=2.5, color=colors[i])

axes[0].set_title('📉 Evolução da Taxa de Aprendizado')
axes[0].set_xlabel('Épocas')
axes[0].set_ylabel('Learning Rate')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Convergência da loss
for i, decay in enumerate(decay_configs):
    _, _, hist = decay_results[decay]
    axes[1].plot(hist['loss'], label=f'Decay = {decay}', 
                linewidth=2.5, color=colors[i])

axes[1].set_title('📈 Convergência com Different Decays')
axes[1].set_xlabel('Épocas')
axes[1].set_ylabel('Loss (MSE)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

print("🎯 Observações sobre Learning Rate Decay:")
print("   • Sem decay (0.0): Pode oscilar no final")
print("   • Decay moderado: Melhora a convergência final")
print("   • Decay alto: Pode ficar muito lento")
print("\n💡 A chave é encontrar o equilíbrio!")

## 🎓 8. Resumo e Lições Aprendidas

Parabéns! 🎉 Você completou o último módulo do nosso curso "Cálculo para IA"! Vamos fazer um resumo épico de tudo que aprendemos sobre otimização:

### 🏔️ A Jornada pela Montanha do Erro:

1. **Gradiente Descendente Batch**: O marombeiro que usa todos os dados
   - ✅ Preciso e estável
   - ❌ Lento para datasets grandes

2. **SGD (Stochastic)**: O apressadinho que usa um exemplo por vez
   - ✅ Rápido e usa pouca memória
   - ❌ Instável, mas pode escapar de mínimos locais

3. **Mini-Batch**: O meio-termo inteligente
   - ✅ Equilibra velocidade e estabilidade
   - ✅ Eficiente para GPUs

4. **Adam**: O algoritmo "espertão" adaptativo
   - ✅ Se adapta automaticamente
   - ✅ Funciona bem na maioria dos casos
   - ✅ Combina momentum + normalização adaptativa

### 🎛️ Sobre a Taxa de Aprendizado:
- **Muito baixa**: Lesma na montanha 🐌
- **Muito alta**: Carro sem freio 🚗💨
- **Na medida certa**: Equilíbrio perfeito! 🎯

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-12_img_08.png)

**Dica Final do Pedro**: Na vida real, comece com Adam e learning rate 0.001. Se não funcionar, aí você experimenta os outros! É como escolher pizza - margherita sempre funciona, mas às vezes você quer experimentar sabores diferentes! 🍕

```mermaid
graph TD
    A[Início do Treinamento] --> B{Escolha do Otimizador}
    B -->|Dataset Pequeno| C[Batch GD]
    B -->|Dataset Grande| D[SGD/Mini-Batch]
    B -->|Uso Geral| E[Adam]
    
    C --> F[Ajustar Learning Rate]
    D --> F
    E --> F
    
    F --> G{Convergiu?}
    G -->|Não| H[Ajustar Hiperparâmetros]
    G -->|Sim| I[🎉 Sucesso!]
    H --> F
```

In [ ]:
# 🏆 COMPARAÇÃO FINAL: Todos os algoritmos que vimos
print("🎊 PARABÉNS! Você completou o curso 'Cálculo para IA'! 🎊\n")
print("📊 RESUMO FINAL - Todos os Otimizadores:")
print("="*60)

# Coletando todos os resultados para comparação final
all_optimizers = {
    'Batch GD': (hist_batch['loss'][-1], abs(hist_batch['w'][-1] - 3.0)),
    'SGD': (hist_sgd['loss'][-1], abs(hist_sgd['w'][-1] - 3.0)),
    'Mini-Batch': (results[32][2]['loss'][-1], abs(results[32][2]['w'][-1] - 3.0)),
    'Momentum': (hist_mom['loss'][-1], abs(hist_mom['w'][-1] - 3.0)),
    'RMSprop': (hist_rms['loss'][-1], abs(hist_rms['w'][-1] - 3.0)),
    'Adam': (hist_adam['loss'][-1], abs(hist_adam['w'][-1] - 3.0))
}

# Se AdaMax foi implementado com sucesso
if 'hist_adamax' in locals():
    all_optimizers['AdaMax'] = (hist_adamax['loss'][-1], abs(hist_adamax['w'][-1] - 3.0))

# Ranking por loss final
sorted_by_loss = sorted(all_optimizers.items(), key=lambda x: x[1][0])

print("\n🏅 RANKING POR LOSS FINAL:")
medals = ['🥇', '🥈', '🥉'] + ['🏆'] * 10
for i, (name, (loss, error)) in enumerate(sorted_by_loss):
    print(f"   {medals[i]} {name:<12}: Loss = {loss:.6f}, Erro_peso = {error:.4f}")

print("\n🎯 RECOMENDAÇÕES PRÁTICAS:")
print("   • Para iniciantes: Comece com Adam (lr=0.001)")
print("   • Para datasets pequenos: Batch GD funciona bem")
print("   • Para datasets grandes: Mini-Batch (32-128)")
print("   • Para pesquisa: Experimente diferentes otimizadores")

print("\n🚀 PRÓXIMOS PASSOS:")
print("   • Aplicar em redes neurais reais")
print("   • Experimentar com datasets reais")
print("   • Estudar regularização e outras técnicas")
print("   • Explorar frameworks como PyTorch/TensorFlow")

print("\n" + "="*60)
print("🎉 OBRIGADO POR FAZER PARTE DESSA JORNADA! 🎉")
print("\n💡 Lembre-se: O cálculo é a linguagem da IA.")
print("    Agora você fala essa linguagem fluentemente!")
print("\n🔥 Vá lá e construa coisas incríveis com IA! 🔥")
print("="*60)

## 🎯 Conclusão: Você Agora é um Otimizador Master!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-12_img_09.png)

### 🏆 O Que Você Conquistou:

✅ **Dominou** as variações do gradiente descendente  
✅ **Entendeu** a importância da taxa de aprendizado  
✅ **Implementou** algoritmos avançados como Adam  
✅ **Comparou** diferentes estratégias de otimização  
✅ **Completou** o curso "Cálculo para IA"!  

### 🌟 Reflexão Final:

Começamos nossa jornada perguntando "Por que se importar com cálculo?". Agora você sabe que o **cálculo é o coração** de toda inteligência artificial moderna!

- **Derivadas** nos mostram a direção certa
- **Gradientes** nos guiam pela montanha do erro  
- **Otimizadores** nos levam ao topo com eficiência

### 🚀 Agora É Sua Vez!

Você tem todas as ferramentas para:
- Criar seus próprios algoritmos de ML
- Entender o que acontece "por debaixo do capô" 
- Debugar e otimizar modelos como um expert
- Contribuir para o avanço da IA!

**Última Dica do Pedro**: O aprendizado nunca para! Continue experimentando, errando, aprendendo e, principalmente, se divertindo com IA. O futuro é nosso para construir! 🌈🤖

---

**Até a próxima aventura!** 🎊  
**Pedro Nunes Guth** 📚✨